<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/answer/12a_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第12章 パターンマイニング

In [1]:
# colabで実行
!pip install -U mlxtend
!pip install scikit-surprise
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 1.3MB 2.7MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675352 sha256=a4e6d90402ff3fe7b79f2771a57d3a5115ec39c98297ffd7c1b21b1cd672d4bc
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

## 課題1

上記の規則抽出のコード例で求めた規則よりも、Lift値の高い規則を探すにはどうすればよいでしょうか。

In [0]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [3]:
!wget https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff

--2020-06-13 07:16:24--  https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2025871 (1.9M) [text/plain]
Saving to: ‘supermarket.arff’

supermarket.arff    100%[===================>]   1.93M  --.-KB/s    in 0.1s    

2020-06-13 07:16:25 (13.7 MB/s) - ‘supermarket.arff’ saved [2025871/2025871]



In [0]:
!sed -e "s/{ t}/{t}/" supermarket.arff > supermarket2.arff

In [5]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket2.arff')
df = pd.DataFrame(data)
df2 = df.replace({b'?':False, b't':True})
df2 = df2.drop('total', axis=1)
df2

,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,department11,baby needs,bread and cake,baking needs,coupons,juice-sat-cord-ms,tea,biscuits,canned fish-meat,canned fruit,canned vegetables,breakfast food,cigs-tobacco pkts,cigarette cartons,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,puddings-deserts,dishcloths-scour,deod-disinfectant,frozen foods,razor blades,fuels-garden aids,spices,jams-spreads,insecticides,pet foods,laundry needs,party snack foods,...,department177,department178,department179,casks white wine,casks red wine,750ml white nz,750ml red nz,750ml white imp,750ml red imp,sparkling nz,sparkling imp,brew kits/accesry,department189,port and sherry,ctrled label wine,department192,department193,department194,department195,department196,department197,department198,department199,non host support,department201,department202,department203,department204,department205,department206,department207,department208,department209,department210,department211,department212,department213,department214,department215,department216
0,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,False,True,False,False,True,False,False,False,True,True,True,True,False,True,False,True,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,True,True,True,False,False,False,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,True,False,False,False,True,False,True,False,True,Fa

FPGrowthでmin_supportを下げて多くの頻出項目集合を求め、association_rulesで規則を評価します。ただしmin_supportをあまり下げすぎると、あまり意味のない項目集合が出てきます。

In [0]:
freq2 = fpgrowth(df2, min_support=0.2, use_colnames=True)

In [7]:
association_rules(freq2, metric="lift", min_threshold=1.5)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(frozen foods, bread and cake, fruit)","(biscuits, vegetables)",0.334558,0.381241,0.200778,0.600129,1.574148,0.073231,1.547398
1,"(bread and cake, frozen foods, vegetables)","(biscuits, fruit)",0.334558,0.397018,0.200778,0.600129,1.511594,0.067953,1.507943
2,"(biscuits, bread and cake, vegetables)","(frozen foods, fruit)",0.321375,0.402204,0.200778,0.624748,1.553309,0.071520,1.593050
3,"(frozen foods, fruit)","(biscuits, bread and cake, vegetables)",0.402204,0.321375,0.200778,0.499194,1.553309,0.071520,1.355067
4,"(biscuits, fruit)","(bread and cake, frozen foods, vegetables)",0.397018,0.334558,0.200778,0.505716,1.511594,0.067953,1.346274
5,"(biscuits, vegetables)","(frozen foods, bread and cake, fruit)",0.381241,0.334558,0.200778,0.526644,1.574148,0.073231,1.405796


## 課題2

教科書p.226の演習問題12-3の手順に従い、MovieLensデータセットに対してscikit-surpriseを用いてNMFを行ってください。余力があれば、ハイパーパラメータを変更して、性能に与える影響を観察してください。

In [0]:
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [0]:
# movielens-100k データを読み込みます。読み込みに承諾(Yと入力)が必要です。
X = Dataset.load_builtin('ml-100k')

In [12]:
# 5-fold CVで、平均平方二乗誤差と平均絶対誤差を表示します。
cross_validate(NMF(), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9679  0.9620  0.9590  0.9584  0.9724  0.9639  0.0054  
MAE (testset)     0.7629  0.7597  0.7515  0.7526  0.7636  0.7581  0.0051  
Fit time          4.10    4.29    4.34    4.44    4.38    4.31    0.11    
Test time         0.10    0.10    0.10    0.10    0.10    0.10    0.00    


{'fit_time': (4.103447198867798,
  4.28737211227417,
  4.33703351020813,
  4.437155246734619,
  4.3767173290252686),
 'test_mae': array([0.76291664, 0.75972147, 0.75145383, 0.75258402, 0.76360532]),
 'test_rmse': array([0.96792385, 0.9619538 , 0.95897283, 0.95841578, 0.97240769]),
 'test_time': (0.10168242454528809,
  0.10147380828857422,
  0.09989213943481445,
  0.10352659225463867,
  0.09964394569396973)}

scikit-surprise NMFのデフォルトの圧縮次元数(n_factors)は15です。これを下げると、実行時間は早くなっていますが、性能は悪くなっています。

In [13]:
cross_validate(NMF(n_factors=5), X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0578  1.0583  1.0659  1.0511  1.0651  1.0596  0.0054  
MAE (testset)     0.8700  0.8701  0.8760  0.8639  0.8763  0.8712  0.0046  
Fit time          3.10    3.09    3.10    2.98    2.90    3.03    0.08    
Test time         0.11    0.10    0.18    0.10    0.10    0.12    0.03    


{'fit_time': (3.095869541168213,
  3.091820478439331,
  3.1009621620178223,
  2.9782252311706543,
  2.9015393257141113),
 'test_mae': array([0.86998376, 0.87010536, 0.8759672 , 0.86390827, 0.87626847]),
 'test_rmse': array([1.05784341, 1.05828221, 1.06588629, 1.05110054, 1.06513617]),
 'test_time': (0.10755014419555664,
  0.10035252571105957,
  0.17710328102111816,
  0.09696531295776367,
  0.09752869606018066)}